<a href="https://colab.research.google.com/github/laukikupadhye/NetflixRecommendation/blob/main/DataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Cleaning and processing for Netflix movie recommender

##Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
data =pd.read_csv("/content/netflix_titles.csv")
data.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


##Converting string to date and creating new column for release year and month

In [ ]:
data['date_added']=pd.to_datetime(data['date_added'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       6234 non-null   int64         
 1   type          6234 non-null   object        
 2   title         6234 non-null   object        
 3   director      4265 non-null   object        
 4   cast          5664 non-null   object        
 5   country       5758 non-null   object        
 6   date_added    6223 non-null   datetime64[ns]
 7   release_year  6234 non-null   int64         
 8   rating        6224 non-null   object        
 9   duration      6234 non-null   object        
 10  listed_in     6234 non-null   object        
 11  description   6234 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 584.6+ KB


In [ ]:
#data['release_year']=data['date_added'].strftime("%Y")
data['release_year']=data['date_added'].dt.year
data['release_month']=data['date_added'].dt.month_name().str.slice(stop=3)
data.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,release_month
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09,2019.0,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,Sep
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,2016-09-09,2016.0,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,Sep
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09-08,2018.0,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",Sep
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09-08,2018.0,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,Sep
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09-08,2017.0,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,Sep


##Creating table for showing count of movies by each year and month

In [ ]:
dataA=data.groupby(['release_year','release_month'],as_index=False).agg(count_movies=('title','count')).copy()
#dataA.head(33121)

In [ ]:
dataA.to_csv('/content/year_month_count.csv', index = False)

In [ ]:
dataA.head()

,release_year,release_month,count_movies
0,2008.0,Feb,1
1,2008.0,Jan,1
2,2009.0,May,1
3,2009.0,Nov,1
4,2010.0,Nov,1


In [ ]:
monthset=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


In [ ]:
dataB=pd.DataFrame(columns=['Years','Months'])
#dataC
#append rows to it, for each year, append all month
rowlist=[]
for year in range(2008,2021):
  for mon in (monthset):
    #print()
    rowlist.append([year,mon])  
dataB=dataB.append(pd.DataFrame(rowlist,columns=['Years','Months']))


In [ ]:
dataB

,Years,Months
0,2008,Jan
1,2008,Feb
2,2008,Mar
3,2008,Apr
4,2008,May
...,...,...
139,2019,Aug
140,2019,Sep
141,2019,Oct
142,2019,Nov


In [ ]:
dataC = pd.merge(left=dataB, right=dataA, how='outer', left_on=['Years','Months'], right_on=['release_year','release_month'])
dataC

,Years,Months,release_year,release_month,count_movies
0,2008,Jan,2008.0,Jan,1.0
1,2008,Feb,2008.0,Feb,1.0
2,2008,Mar,NaN,NaN,NaN
3,2008,Apr,NaN,NaN,NaN
4,2008,May,NaN,NaN,NaN
...,...,...,...,...,...
151,2020,Aug,NaN,NaN,NaN
152,2020,Sep,NaN,NaN,NaN
153,2020,Oct,NaN,NaN,NaN
154,2020,Nov,NaN,NaN,NaN


In [ ]:
dataC['count_movies']=dataC[(dataC['count_movies']==NaN)]

In [ ]:
dataC[['Years','Months','count_movies']].to_csv('/content/year_month_count.csv', index = False)

#Find top 5 countries and top two movies each for nested pie chart

In [ ]:
data =pd.read_csv("/content/netflix_IMDB.csv")
data['date_added']=pd.to_datetime(data['date_added'])
data['release_year']=data['date_added'].dt.year
data['release_month']=data['date_added'].dt.month_name().str.slice(stop=3)

In [ ]:
data.info()

In [ ]:
#Count by year and month
dataA=data.groupby(['release_year','release_month','country_x'],as_index=False).agg(count_movies=('title_x','count')).copy()

In [ ]:
dataA

In [ ]:
dataPieOuter=dataA[(dataA.release_year==2020) &(dataA.release_month=='Jan')].sort_values(by='count_movies',ascending=False).head(5).copy()
dataPieOuter.to_csv('/content/country_wise_count.csv', index = False)

year month wise top 5 countryies

In [ ]:
dataPieOuter1=dataA.sort_values(by='count_movies',ascending=False).copy()
#dataPieOuter.to_csv('/content/country_wise_count.csv', index = False)
dataAllYearsPieOuter=dataPieOuter1.groupby(['release_year','release_month']).head(5)
dataAllYearsPieOuter.to_csv('/content/country_wise_count_all_years.csv', index = False)

In [ ]:
dataAllYearsPieOuter

In [ ]:
dataC=data[['country_x','title_x','avg_vote']][(data.country_x.isin(dataPieOuter.country_x.to_list()))].sort_values(by='avg_vote',ascending=False)

In [ ]:
dataPieInner=dataC.groupby('country_x').head(2)
dataPieInner.to_csv('/content/country_movies_ratings.csv',index=False)

Adding outerpie data values to innerpie data

In [ ]:
newPieInner = pd.merge(left=dataPieInner, right=dataPieOuter, how='left', left_on=['country_x'], right_on=['country_x'])

In [ ]:
newPieInner=newPieInner[['country_x','title_x','count_movies','avg_vote']]
newPieInner.to_csv('/content/country_movies_ratings.csv',index=False)

In [ ]:
newPieInner

New Inner pie data

In [ ]:
dataP=data[['release_year','release_month','country_x','title_x','avg_vote']].sort_values(by='avg_vote',ascending=False)

In [ ]:
newInnerPie=dataP.groupby(['release_year','release_month','country_x']).head(2)
#newInnerPie[(dataP.release_year==2020)&(dataP.release_month=='Jan')&(dataP.country_x=='India')]
#newInnerPie.to_csv('/content/country_movie_rating.csv',index=False)

In [ ]:
newInnerPie

In [ ]:
innerDataStore=pd.merge(left=dataAllYearsPieOuter, right=newInnerPie, how='left', left_on=['release_year','release_month','country_x'], right_on=['release_year','release_month','country_x'])
#innerDataStore[(innerDataStore.release_year==2020)&(innerDataStore.release_month=='Jan')&(innerDataStore.country_x=='India')]
innerDataStore.to_csv('/content/country_movie_ratings.csv',index=False)

In [ ]:
innerDataStore[(innerDataStore.release_year==2020)&(innerDataStore.release_month=='Jan')&(innerDataStore.country_x=='India')]

#Movie Recommendation by director

In [ ]:
director=data[(data.title_x.isin(newPieInner.title_x.head(1)))].director_x.values[0]
director

top10Movies=data[(data.director_x==director)][['director_x','title_x','avg_vote']].sort_values(by='avg_vote',ascending=False).head(11).copy()
top10Movies.tail(len(top10Movies)-1).to_csv('/content/movies_by_director.csv',index=False)

#Visesh's Data - for group by

In [ ]:
data =pd.read_csv("/content/map_data.csv")
dataA=data.groupby(['country'],as_index=False).agg(sum_all_years=('count','sum')).copy()
dataA.to_csv('/content/allyearsum_data.csv',index=False)

#Alekhya

In [ ]:
data =pd.read_csv("/content/boxoffice_world_map.csv")
dataA=data.groupby(['title_year','country'],as_index=False).agg(sum_budget=('budget','sum')).copy()
dataA.to_csv('/content/year_contry_budget.csv',index=False)

#week 11 

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
data1 =pd.read_csv("/content/2008-2016.csv")
data2=pd.read_csv("/content/2020.csv")
dataJoin=pd.merge(left=data1, right=data2, how='outer', left_on=['fips_code'], right_on=['county_fips'])

In [ ]:
newData=dataJoin.groupby('state_name',as_index=False).agg(sum).copy()

In [ ]:
newData.to_csv("/content/state_data.csv")